In [1]:
import torch
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2
from pathlib import Path
import wandb

import sys

sys.path.append("../src")

from trainer import Trainer
from module import VGGNetModule
from utils import model_size
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
logger = WandbLogger(
    project_name="ImageClassification",
    config={
        "model_architecture": "VGGNet",
        "batch_size": 64,
        "max_epochs": 100,
        "optimizer": {
            "name": "Adam",
        },
        "train_split": 42_000,
        "val_split": 8000
    },
    logs_path=logs_path
)

In [4]:
dataset = CIFAR10(data_path, train=True, download=True, transform=v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
]))

train_dataset, val_dataset = random_split(
    dataset, [logger.config["train_split"], logger.config["val_split"]])

train_dataloader = DataLoader(
    train_dataset, batch_size=logger.config["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=logger.config["batch_size"])

Files already downloaded and verified


In [5]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=80.0, accuracy_diff=4.0, wait_epochs=5),
    OverfitCallback(limit_batches=2, max_epochs=200),
]

In [6]:
module = VGGNetModule()

trainer = Trainer(
    module=module,
    logger=logger,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=False,
    measure_time=True
)

model_size(module.model)

model size: 0.02 MB


In [7]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


Time per epoch: 65.42 seconds
Epoch: 0, train_accuracy: 35.21, val_accuracy: 45.91
Epoch: 1, train_accuracy: 49.45, val_accuracy: 52.30
Epoch: 2, train_accuracy: 53.72, val_accuracy: 55.91
Epoch: 3, train_accuracy: 56.54, val_accuracy: 57.12
Epoch: 4, train_accuracy: 58.76, val_accuracy: 59.39
Epoch: 5, train_accuracy: 60.07, val_accuracy: 59.70
Epoch: 6, train_accuracy: 61.91, val_accuracy: 62.10
Epoch: 7, train_accuracy: 62.51, val_accuracy: 62.19
Epoch: 8, train_accuracy: 63.37, val_accuracy: 62.56
Epoch: 9, train_accuracy: 64.17, val_accuracy: 63.91
Epoch: 10, train_accuracy: 64.80, val_accuracy: 62.69
Epoch: 11, train_accuracy: 65.17, val_accuracy: 63.86
Epoch: 12, train_accuracy: 65.73, val_accuracy: 64.28
Epoch: 13, train_accuracy: 65.90, val_accuracy: 65.89
Epoch: 14, train_accuracy: 66.55, val_accuracy: 65.70
Epoch: 15, train_accuracy: 66.74, val_accuracy: 64.49
Epoch: 16, train_accuracy: 67.28, val_accuracy: 65.11
Epoch: 17, train_accuracy: 67.53, val_accuracy: 65.29
Epoch: 1

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
epoch_train_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇███████
epoch_train_loss,█▅▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch_val_accuracy,▁▃▅▅▆▆▇▇▇▇▇▇▇██████
epoch_val_loss,█▆▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▂
step,▄▅▆▃▅▄▅▁▂▃▃▅▇█▁▂▅▅▅▆▁▁▄▅▂▄▃▇▂▄▁▃▆▁▂▆▄▂▂▂
epoch,18
epoch_train_accuracy,67.64175
epoch_train_loss,0.89915
epoch_val_accuracy,65.3375
epoch_val_loss,0.99319


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)